In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from qamsi.config.trading_config import TradingConfig
from qamsi.runner import Runner
from qamsi.strategies.estimated.min_var import MinVariance
from qamsi.cov_estimators.cov_estimators import CovEstimators
from qamsi.features.preprocessor import Preprocessor
from run import Dataset

(CVXPY) Jun 11 06:35:04 PM: Encountered unexpected exception importing solver PROXQP:
ImportError("dlopen(/Users/buchkovv/qamsi/.venv/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/proxsuite_pywrap.cpython-312-darwin.so, 0x0002): Library not loaded: @rpath/libc++.1.dylib\n  Referenced from: <73C5C23C-530A-3E0E-A88E-7897A0C69618> /Users/buchkovv/qamsi/.venv/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/proxsuite_pywrap.cpython-312-darwin.so\n  Reason: tried: '/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/var/folders/0j/bwqcs4y508s2n4ck4dhf3rpc0000gn/T/cmeel-qko5evau/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/var/folders/0j/bwqcs4y508s2n4ck4dhf3rpc0000gn/T/cmeel-qko5evau/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/Users

In [3]:
REBAL_FREQ = "ME"
DATASET = Dataset.SPX_US
ESTIMATION_WINDOW = 365 * 1

ESTIMATOR = CovEstimators.STATIC_FACTOR_MODEL.value(
    factor_cov_estimator=CovEstimators.HISTORICAL.value(),
    residual_cov_estimator=CovEstimators.SHRINKAGE_PCA.value(),
)

In [4]:
experiment_config = DATASET.value()

stocks = tuple(
    pd.read_csv(experiment_config.PATH_OUTPUT / experiment_config.STOCKS_LIST_FILENAME)
    .iloc[:, 0]
    .astype(str)
    .tolist(),
)
experiment_config.ASSET_UNIVERSE = stocks  # type: ignore  # noqa: PGH003

experiment_config.MIN_ROLLING_PERIODS = ESTIMATION_WINDOW + 1
experiment_config.N_LOOKBEHIND_PERIODS = None
experiment_config.REBALANCE_FREQ = REBAL_FREQ

factors = pd.read_csv(experiment_config.PATH_OUTPUT / "factors.csv")
factors["date"] = pd.to_datetime(factors["date"])
factors = factors.set_index("date")
factor_names = tuple(factors.columns.astype(str).tolist())
experiment_config.FACTORS = factor_names

prices = [stock + "_Price" for stock in list(stocks)]
preprocessor = Preprocessor(
    exclude_names=[
        *list(stocks),
        experiment_config.RF_NAME,
        *experiment_config.HEDGING_ASSETS,
        *factor_names,
        *prices,
    ],
)

trading_config = TradingConfig(
    broker_fee=0.05 / 100,
    bid_ask_spread=0.03 / 100,
    total_exposure=1,
    max_exposure=1,
    min_exposure=0,
    trading_lag_days=1,
)

runner = Runner(
    experiment_config=experiment_config,
    trading_config=trading_config,
    verbose=True,
)

Backtest on 2004-03-19 00:00:00 to 2023-12-29 00:00:00


In [5]:
strategy = MinVariance(
    cov_estimator=ESTIMATOR,
    trading_config=trading_config,
    window_size=ESTIMATION_WINDOW,
)

strategy_name = ESTIMATOR.__class__.__name__

In [15]:
full_ret = runner.returns.simple_returns.sub(runner.rf, axis=0)

YEAR = "2008"

ret = full_ret.loc[f"{YEAR}-01-01" : f"{YEAR}-12-31"]
factors = runner.factors.loc[f"{YEAR}-01-01" : f"{YEAR}-12-31"]
feat = runner.features.loc[f"{YEAR}-01-01" : f"{YEAR}-12-31"]

n_valid_points = (~ret.isna()).sum(axis=0) / len(ret)
valid_stocks = list(n_valid_points[n_valid_points >= 1.0].index)

ret = ret[valid_stocks]

In [10]:
from qamsi.strategies.optimization_data import TrainingData, PredictionData

train = TrainingData(
    simple_excess_returns=ret,
    factors=factors,
    features=feat.iloc[:-1],
)
pred = PredictionData(
    features=feat,
)

In [17]:
strategy.fit(train)

In [18]:
w = strategy.get_weights(pred)
w.shape

AttributeError: 'NoneType' object has no attribute 'index'

In [16]:
realized_ret = full_ret.loc[ret.index.max() + pd.Timedelta(days=1) :].iloc[:21]
realized_ret.shape

(21, 1033)

In [13]:
runner(
    feature_processor=preprocessor,
    strategy=strategy,
)

Computing Weights: 100%|██████████| 221/221 [02:25<00:00,  1.52it/s]


StrategyStatistics:
* final_nav = 3.702012
* geom_avg_total_r = 0.092541
* geom_avg_xs_r = 0.057501
* std_xs_r = 0.139489
* min_xs_r = -0.085674
* max_xs_r = 0.089431
* skew = -0.137467
* kurtosis = 20.392237
* max_dd = -0.310051
* sharpe = 0.412225
* alpha_buy_hold = -0.004748
* tracking_error_buy_hold = 0.143017
* ir_buy_hold = -0.033199
* factor_loadings = {'low_risk': 0.48878135637228004, 'momentum': -0.00863650410739029, 'quality': -0.027654241695449267, 'size': 0.09794482176001071, 'value': -0.1143133287338174, 'spx': 0.6739049913007401}
* alpha_benchmark = 0.007223
* alpha_benchmark_pvalue = 0.634304
* tracking_error_benchmark = 0.078309
* ir_benchmark = 0.092243
* ttest_pval = 0.609001
* levene_pval = 0.000000
* timing_ability_coef = -0.027671
* timing_ability_pval = 0.075862